In [1]:
import pandas as pd
from openai import OpenAI
import numpy as np
import random
import json


In [2]:
train_solutions = pd.read_excel('data/for_teams/train/solutions.xlsx')
train_tasks = pd.read_excel('data/for_teams/train/tasks.xlsx')
train_tests = pd.read_excel('data/for_teams/train/tests.xlsx')

test_solutions = pd.read_excel('data/for_teams/test/solutions.xlsx')
test_tasks = pd.read_excel('data/for_teams/test/tasks.xlsx')
test_tests = pd.read_excel('data/for_teams/test/tests.xlsx')

In [3]:
def combine_data(solutions_, tasks_, tests_):
    solutions, tasks, tests = solutions_.copy(), tasks_.copy(), tests_.copy()
    
    grouped_tests = tests.groupby('task_id').apply(
        lambda x: x[['number', 'type', 'input', 'output']].to_dict('records')
    ).reset_index()

    grouped_tests = grouped_tests.rename(columns={0: 'tests'})
    
    merged_df = pd.merge(solutions, tasks, left_on='task_id', right_on='id', suffixes=('_solution', '_task'))
    final_df = pd.merge(merged_df, grouped_tests, how='left', on='task_id')
    
    return final_df

In [4]:
train = combine_data(train_solutions, train_tasks, train_tests)
test = combine_data(test_solutions, test_tasks, test_tests)

/tmp/ipykernel_299515/1392421440.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped_tests = tests.groupby('task_id').apply(
/tmp/ipykernel_299515/1392421440.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped_tests = tests.groupby('task_id').apply(


In [5]:
i = 0
print(train.iloc[i].description)
print("---------------------------------")
print(train.iloc[i].author_solution)
print("---------------------------------")
print(train.iloc[i].student_solution)
print("---------------------------------")
print(train.iloc[i].author_comment)

Реализуйте программу, которая проверит, что цвет используется только в проекте по созданию логотипа, но не в проекте по созданию дизайна сайта:

Даны два списка logo_project и cite_project с кодами используемых цветов (строки).
В переменную color считывается код цвета (строка). Этот код уже написан.
Программа должна проверять, что код цвета color есть только в списке logo_project, и если да, то печатать True. 
В остальных случаях программа печатает False. 
---------------------------------
logo_project = ['#a7a8f0', '#a7f0ca', '#b3b4e4', '#e4b3cd', '#e4e3b3', '#c0ced7']
cite_project = ['#e4e3b3', '#a7a8f0', '#ccb1e6', '#b4f99e', '#f9b59e', '#c0ced7']

color = input()

if color in logo_project and not(color in cite_project):
    print(True)
else:
    print(False)
---------------------------------
logo_project = ['#a7a8f0', '#a7f0ca', '#b3b4e4', '#e4b3cd', '#e4e3b3', '#c0ced7']
cite_project = ['#e4e3b3', '#a7a8f0', '#ccb1e6', '#b4f99e', '#f9b59e', '#c0ced7']

color = input()

if color in

In [8]:
train.groupby(by="task_id").count()['student_solution']

task_id
1     14
2     16
3     17
6     17
7     16
14    15
18    17
21    16
23    21
26    14
27    17
28    14
29    15
30     9
32    18
34    16
35    24
38    15
39    12
40    13
43    15
47    16
Name: student_solution, dtype: int64

In [9]:
train.task_id.unique()

array([ 1,  2,  3,  6,  7, 14, 18, 21, 23, 26, 27, 28, 29, 30, 32, 34, 35,
       38, 39, 40, 43, 47])

In [10]:
test.task_id.unique()

array([ 0,  4,  5, 12, 13, 15, 16, 17, 19, 20, 22, 24, 25, 31, 33, 36, 37,
       41, 42, 44, 45, 46])

In [11]:
tasks = train.description.unique()

In [12]:
client = OpenAI(
	api_key="secret"
)


def get_llm_answer(mes, client=client):
    messages = [
    	{"role": "user", "content": mes}
    ]
    
    completion = client.chat.completions.create(
        model="gpt-4o", 
    	messages=messages
    )
    
    return completion.choices[0].message.content

In [13]:
get_llm_answer("Привет")

'Привет! Как я могу помочь вам сегодня?'

# Генерируем новые задачи, сразу с правильным решением и тестами

In [34]:
def get_new_ans(tasks):
    promt = f"""Ты - профессиональный преподаватель-методист по программированию на python. Ты учишь студентов этому языку программирования.
    
    Тебе нужно придумать задачу, которая проверяет знания синтаксиса языка, похожую на данные снизу.
    Задача должна принимать что-то на вход, и давать вывод, то есть использовать input и print.
    Сделай задачу похожую по уровню сложности, но при этом разнообразную и отличающуюся от задач ниже:
    {tasks}
    В ответе я жду: 
    - Текст задачи
    - Код для ее решения на python
    - 3 примера входных данных для тестов
    - 3 output для них для входных данных, которые должны проверять правильность кода
    Не пиши комментариев в коде! Не пиши лишних символов, ковычек! Я жду от тебя только json, который я буду парсить. Заменяй все \n на \\n
    Ответ жду в формате json:
{{"task": "Текст задачи", "code": "Код задачи", "inputs": ["input1", "input2", "input3"], "outputs": ["output1", "output2", "output3"]}}
    Ответ:"""
    ans = get_llm_answer(promt)
    ans = ans.replace("`", '')
    if ans[:4] == "json": ans = ans[4:]
    print(ans)
    data = json.loads(ans)
    return data

In [36]:
def get_new_task(tasks):
    tsk = np.random.choice(tasks, 5)
    tsk_str = ''
    for t in tsk:
        tsk_str += t + '\n\n'
    return get_new_ans(tsk_str)

In [37]:
new_task = get_new_task(tasks)
print(new_task)


{
    "task": "Полина готовится к экзамену по программированию на Python и хочет проверить свои знания о использовании списка и функции sum. Напишите программу, которая находит максимальную сумму соседних отличных от нуля чисел из списка. \n\nФОРМАТ ВВОДА\nНа вход программе подается строка целых чисел, записанных через пробел. Гарантируется, что хотя бы одно число в списке отлично от нуля.\n\nФОРМАТ ВЫВОДА\nОдно целое число — максимальная сумма соседних отличных от нуля чисел.",
    "code": "s = list(map(int, input().split()))\nmax_sum = 0\ncurrent_sum = 0\nfor num in s:\n    if num != 0:\n        current_sum += num\n    else:\n        max_sum = max(max_sum, current_sum)\n        current_sum = 0\nmax_sum = max(max_sum, current_sum)\nprint(max_sum)",
    "inputs": ["1 2 0 3 4 0 0 5 6", "0 0 7 8 9 0 0 1 0", "1 0 2 0 3"],
    "outputs": ["7", "24", "3"]
}

{'task': 'Полина готовится к экзамену по программированию на Python и хочет проверить свои знания о использовании списка и функции su

In [40]:
new_tasks = pd.DataFrame({
  "task": [],
  "code": [],
  "inputs": [],
  "outputs": []
})
new_tasks

,task,code,inputs,outputs


In [46]:
new_tasks.loc[len(new_tasks)] = new_task
new_tasks

,task,code,inputs,outputs
0,Полина готовится к экзамену по программировани...,"s = list(map(int, input().split()))\nmax_sum =...","[1 2 0 3 4 0 0 5 6, 0 0 7 8 9 0 0 1 0, 1 0 2 0 3]","[7, 24, 3]"


In [47]:
for i in range(100):
  try:
    new_task = get_new_task(tasks)
    print(i, new_task)
    new_tasks.loc[len(new_tasks)] = new_task
  except:
    pass
  


{
    "task": "Реализуйте программу, которая проверяет строки на валидность по заданному шаблону.\nВам нужно прочитать строки с консоли, которые будут вводиться в цикле до ввода 'СТОП'.\nКаждая строка считается валидной, если она содержит только символы нижнего регистра и пробел.\nПри этом количество слов (разделенных пробелом) в этих строках должно быть четным.\nНеобходимо вывести количество валидных строк. Гарантируется, что до слова 'СТОП' будет введена как минимум одна строка.",
    "code": "count = 0\nwhile True:\n    line = input()\n    if line == 'СТОП':\n        break\n    words = line.split()\n    if all(word.islower() for word in words) and len(words) % 2 == 0:\n        count += 1\nprint(count)",
    "inputs": [
        "hello world\npython code\nСТОП",
        "one two three\nok\nСТОП",
        "test\ncase\nexam pass\nСТОП"
    ],
    "outputs": [
        "1",
        "0",
        "1"
    ]
}

0 {'task': "Реализуйте программу, которая проверяет строки на валидность по задан

In [48]:
new_tasks

,task,code,inputs,outputs
0,Полина готовится к экзамену по программировани...,"s = list(map(int, input().split()))\nmax_sum =...","[1 2 0 3 4 0 0 5 6, 0 0 7 8 9 0 0 1 0, 1 0 2 0 3]","[7, 24, 3]"
1,"Реализуйте программу, которая проверяет строки...",count = 0\nwhile True:\n line = input()\n ...,"[hello world\npython code\nСТОП, one two three...","[1, 0, 1]"
2,"Дан текстовый файл words.txt, содержащий набор...","vowels = 'aeiouAEIOU'\nwith open('words.txt', ...",[Beautiful enchanting nature awaits you within...,"[enchanting, Beautiful, possibilities, adventu..."
3,"Реализуйте программу, которая проверяет, может...","def check_access(department, employee_id):\n ...","[research\nR123, finance\nF555, development\nD...","[True, False, True]"
4,"Реализуйте программу, которая проверяет, являю...","words = input().split(', ')\npalindromes = [wo...","[radar, level, cat, civic, dog, apple, banana,...","[civic, level, radar, madam, racecar, rotor, ]"
...,...,...,...,...
95,"Максим решает, каким блогерам стоит обратиться...","filename = 'bloggers.txt'\nwith open(filename,...",[This is a test file @blogger1 mentioned some ...,"[@blogger2, @blogger5, @someone]"
96,Василий организует домашнюю библиотеку и хочет...,book_statuses = input().split('\n')\nold_books...,"[Древо жизни, хорошее\nСтарый дом, обветшалое\...","[Старый дом, Земля и небо, Основание, Книг с о..."
97,"Николай хочет узнать, на какую сумму скидок он...","stores_discounts = input().split(', ')\nmax_di...","[Магазин1: 10%, Магазин2: 15%, Магазин3: 15%, ...","[Магазин2, Магазин3, БутикА, Shop1, Shop3]"
98,"Никита увлекается книгами и решил выяснить, ск...",n = int(input())\nauthors = [input().strip() f...,"[5\nТолстой\nТолстой\nЧехов\nЧехов\nЧехов, 7\n...","[Чехов, Толстой, Гоголь]"


In [49]:
new_tasks.to_csv("data/generated_tasks.csv")